### Explore staging/i94_data.parquet

Exploration summary:
- 40,790,529 records
- 28 columns
- Numeric columns all typed as doubles
- i94res has no missing values
- i94bir has suspicious values, e.g. age < 0 and age >> 100

Cleaning steps needed:
- Keep columns of interest (i94mon, i94res, i94mode, i94bir, i94visa, visatype)
- Cast double typed columns as ints (i94mon, i94res, i94mode, i94bir, i94visa)
- Decode selected columns using mapping file (i94res)
- Include country_fk column from mapping file
- Decode selected columns using F.when (i94mode, i94visa, visatype)
- Replace any suspcious ages with null

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Get SparkSession object
spark = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Filename of input file
filename = 's3://data-eng-capstone-cf/staging/i94_data'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Read into a spark dataframe
df = spark.read.parquet(filename)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Print schema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [7]:
# What does the file look like?
df.select('cicid', 'i94mon', 'i94cit', 'i94res', 'gender').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+------+------+
|cicid    |i94mon|i94cit|i94res|gender|
+---------+------+------+------+------+
|3500517.0|11.0  |135.0 |135.0 |F     |
|3472340.0|11.0  |135.0 |135.0 |M     |
|3477995.0|11.0  |135.0 |135.0 |F     |
|3516439.0|11.0  |135.0 |135.0 |M     |
|3521735.0|11.0  |135.0 |135.0 |F     |
|3524323.0|11.0  |135.0 |135.0 |M     |
|3504360.0|11.0  |135.0 |135.0 |M     |
|3511273.0|11.0  |135.0 |135.0 |F     |
|3517813.0|11.0  |135.0 |135.0 |F     |
|3722246.0|11.0  |135.0 |135.0 |F     |
|3693725.0|11.0  |135.0 |135.0 |M     |
|3722337.0|11.0  |135.0 |135.0 |M     |
|3690868.0|11.0  |135.0 |135.0 |F     |
|3728162.0|11.0  |135.0 |135.0 |M     |
|3693974.0|11.0  |135.0 |135.0 |F     |
|3691708.0|11.0  |135.0 |135.0 |M     |
|3697446.0|11.0  |135.0 |135.0 |F     |
|3880708.0|11.0  |135.0 |135.0 |M     |
|3887896.0|11.0  |135.0 |135.0 |F     |
|3881010.0|11.0  |135.0 |135.0 |F     |
+---------+------+------+------+------+
only showing top 20 rows

In [8]:
# How many records?
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

40790529

In [14]:
# How many columns?
len(df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

28

In [13]:
# How many NaN/None valued rows are there per column?
# https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
df.select([F.sum((F.isnan(c) | F.col(c).isNull()).cast(IntegerType())).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+--------+-------+-------+--------+-------+-------+-------+-------+--------+-------+------+------+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|   occup|entdepa|entdepd| entdepu|matflag|biryear|dtaddto| gender|  insnum|airline|admnum| fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+--------+-------+-------+--------+-------+-------+-------+-------+--------+-------+------+------+--------+
|    0|    0|     0| 28575|     0|      0|      0|  73949|2027926|3308012|  9517|      0|    0|  131050|24032175|40597574|   2404|3287909|40777323|3219581|   9517| 101551|4079983|35678095|1308066|     0|333922|       0|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------

In [15]:
# Check if unqiue in cicid
df.select('cicid').count() == df.select('cicid').distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False

In [19]:
# What are unique values in i94mode?
df.groupby('i94mode').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
|i94mode|   count|
+-------+--------+
|    0.0|       4|
|   null|   73949|
|    1.0|39166088|
|    3.0| 1095001|
|    2.0|  387184|
|    9.0|   68303|
+-------+--------+

In [20]:
# What are unique values in i94visa?
df.groupby('i94visa').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
|i94visa|   count|
+-------+--------+
|    1.0| 5575279|
|    3.0| 1573271|
|    2.0|33641979|
+-------+--------+

In [21]:
# What are unique values in i94visa?
df.groupby('visatype').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+
|visatype|   count|
+--------+--------+
|      F2|   68866|
|     GMB|    2728|
|      B2|15188785|
|      F1| 1487432|
|     CPL|     236|
|      I1|    2825|
|      WB| 2940456|
|      M1|   16306|
|      B1| 2282096|
|      WT|16915615|
|      M2|     667|
|      CP|  272007|
|     GMT| 1265275|
|      E1|   48905|
|       I|   39054|
|      E2|  259215|
|     SBP|      61|
+--------+--------+

In [22]:
# What is the average age?
df.select('i94bir').describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|           i94bir|
+-------+-----------------+
|  count|         40781012|
|   mean|39.53764980133401|
| stddev| 17.8102508817733|
|    min|             -3.0|
|    max|           1812.0|
+-------+-----------------+